# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

This notebook will test production-critical “long-term memory via consolidation” behavior, i.e.:

session history (user/assistant only) → LLM consolidation → UserProfile.memory_entries

UserProfile.memory_entries → LLM generation → UserProfile.system_instructions

persisted profile update → session marked needs_user_instructions_refresh=True

refresh path: SessionManager.get_user_profile_instructions_for_session(session) updates snapshot + clears flag

In [2]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# Notebook 09 - Cell 1 (CONFIG)
# Goal: one-cell environment bootstrap, using ONLY classes that exist in INTERGRAX_ENGINE_BUNDLE.py.

import os
from typing import List

from intergrax.llm.messages import ChatMessage
from intergrax.memory.stores.in_memory_conversational_store import InMemoryConversationalMemoryStore
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.runtime.user_profile.user_profile_instructions_service import UserProfileInstructionsConfig, UserProfileInstructionsService

# Runtime-level consolidation (session -> profile memory entries)
from intergrax.runtime.user_profile.session_memory_consolidation_service import SessionMemoryConsolidationConfig, SessionMemoryConsolidationService

# LLM adapter (real adapter; uses defaults from the adapter itself)
from intergrax.llm_adapters.ollama_adapter import LangChainOllamaAdapter

# ----------------------------
# Identifiers
# ----------------------------
USER_ID = "user_artur"
SESSION_ID = "sess_notebook09_001"

# ----------------------------
# Stores / managers (real framework components)
# ----------------------------
conversational_store = InMemoryConversationalMemoryStore()
user_profile_store = InMemoryUserProfileStore()
user_profile_manager = UserProfileManager(store=user_profile_store)

# ----------------------------
# LLM adapter (real adapter)
# ----------------------------
llm = LangChainOllamaAdapter()

# ----------------------------
# Consolidation service under test
# ----------------------------

instructions_cfg = UserProfileInstructionsConfig(
    regenerate_if_present=False,
)

consolidation_cfg = SessionMemoryConsolidationConfig(
    regenerate_system_instructions=True,
    force_regenerate_system_instructions=False,
)

instructions_service = UserProfileInstructionsService(
    llm=llm,
    manager=user_profile_manager,
    config=instructions_cfg,
)

session_memory_consolidation_service = SessionMemoryConsolidationService(
    llm=llm,
    profile_manager=user_profile_manager,
    instructions_service=instructions_service,
    config=consolidation_cfg,
)

# ----------------------------
# Seed baseline-style history (user/assistant only)
# We'll use this later to trigger consolidation into UserProfile.memory_entries.
# ----------------------------
seed_history: List[ChatMessage] = [
    ChatMessage(role="user", content="I am Artur. I build Intergrax and Mooff. I prefer concise technical answers."),
    ChatMessage(role="assistant", content="Understood. I will keep responses technical and concise."),
    ChatMessage(role="user", content="Never use emojis in code or technical docs. Default project context: Intergrax Drop-In Knowledge Runtime."),
    ChatMessage(role="assistant", content="Acknowledged. No emojis in code/docs; default context set to Intergrax Drop-In Knowledge Runtime."),
]

async def _bootstrap() -> None:
    # Persist seeded conversation into the ConversationalMemoryStore
    memory = await conversational_store.load_memory(session_id=SESSION_ID)
    for m in seed_history:
        await conversational_store.append_message(memory=memory, message=m)

    # Ensure user profile exists
    profile = await user_profile_store.get_profile(USER_ID)

    print("CONFIG READY")
    print(f"- USER_ID   : {USER_ID}")
    print(f"- SESSION_ID: {SESSION_ID}")
    print(f"- seeded_messages: {len(seed_history)}")
    print(f"- profile_exists  : {profile.identity.user_id == USER_ID}")

await _bootstrap()



CONFIG READY
- USER_ID   : user_artur
- SESSION_ID: sess_notebook09_001
- seeded_messages: 4
- profile_exists  : True
